In [110]:
yoi = [2020]
toi = ['15RTN', '15RTP', '14RNS']
root_path = "C:/Users/mcecil/CGA/CDL/"
spath = root_path + f"CDL_HLS_dataframe{yoi[0]}.csv"
url_file = root_path + f'CDL_HLS_dataframe{yoi[0]}.csv'
index_file = root_path + f'mask_index_{yoi[0]}.csv'
kml_file = root_path + 'sentinel_tile_grid.kml'
geojson_file = root_path + 'aoi.geojson'  ## chip file, lat-long

geojson_dir = root_path + 'geojson/'

image_index_file = root_path + f'image_index_{yoi[0]}.csv'


geojson_rpj_file = root_path + 'aoi_rpj.geojson'
hdf_dir = root_path + 'hdf/'
tiff_dir = root_path + 'tif/'
tiff_dir_rpj = root_path + 'tif_5070/'

mask_dir = root_path + 'mask/'

tile_src_path = root_path + "tile_src.csv"

# New tile edge length (in pixels)
npixels = 512

# Directory containing scenes
scdir = root_path + 'scenes/'

# Directory containing masks
madir = root_path +  'masks/'

# Location to save new files
svdir = root_path +  'subsetted_USSE/'


In [85]:

# Switch plotting backend
import matplotlib
matplotlib.use('agg')

# Import required modules
#import cartopy.crs as ccrs
import os
from datetime import datetime
import pandas as pd
import geopandas
#import pyhdf
import rasterio
import rasterio.mask
import matplotlib.pyplot as pp
import nasa_hls
import numpy as np
import time
import fiona
from pathlib import Path
import time
import urllib.request as urlreq
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [ ]:
# Switch plotting backend
import matplotlib
matplotlib.use('agg')


### Import modules
from glob import glob
import numpy as np
import matplotlib.pyplot as pp
import rasterio

In [ ]:
from rasterio.windows import Window

### Locate mask files and loop over them
masks = sorted(glob(madir+'*.tif'))
#print(masks)

Adapted code from burn scar workflow (not working)

In [ ]:
# from rasterio.windows import Window

# # Scale value for HLS S30 bands
# scale = 0.0001

# ### Locate mask files and loop over them
# masks = sorted(glob(madir+'*.tif'))

# for mask in masks[0:10]:
#     print(mask)
#     mask = mask.replace('\\', '/')
#     # Read data
#     src = rasterio.open(mask)
#     data = src.read(1)
    
#     # Skip if no burn scar
#     if (np.sum(data == 1) == 0):
#         print('skipped')
#         continue
    
#     # Locate burn scar center point
#     xinds = np.arange(data.shape[0])
#     yinds = np.arange(data.shape[1])
#     xg, yg = np.meshgrid(xinds, yinds)
#     xc = np.floor(np.mean(xg[data==1]))
#     yc = np.floor(np.mean(yg[data==1]))
    
#     # Compute edges of new box
#     x1 = int(xc-npixels//2)
#     x2 = int(xc+npixels//2)
#     y1 = int(yc-npixels//2)
#     y2 = int(yc+npixels//2)
    
#     # Check if new inds are out of bounds
#     if (x2 > data.shape[0]+1):
#         shift = x2-((data.shape[0]+1))+1
#         x2 = x2 - shift
#         x1 = x1 - shift
        
#     if (x1 < 0):
#         shift = abs(x1)+2
#         x1 = x1 + shift
#         x2 = x2 + shift
        
#     if (y2 > data.shape[1]+1):
#         shift = (y2-(data.shape[1]+1))+1
#         y2 = y2 - shift
#         y1 = y1 - shift
        
#     if (y1 < 0):
#         shift = abs(y1)+2
#         y1 = y1 + shift
#         y2 = y2 + shift

#     # Subset the mask
#     subset_data = np.expand_dims(data[y1:y2, x1:x2], axis=0)
            
#     ### Grab matching scene files
#     name = mask.replace('\\', '/').split("/")[-1].replace(".mask.tif", "")
#     print(name)

#     scene_dir = f'{tiff_dir}{name}'
#     print(scene_dir)
#    # scene_dir = f'{scdir}'

#     ### Comment out unwanted bands
#     #src1 = rasterio.open(f'{scene_dir}/{name}__Coastal_Aerosol.tif')
#     src2 = rasterio.open(f'{scene_dir}/{name}__Blue.tif')
#     src3 = rasterio.open(f'{scene_dir}/{name}__Green.tif')
#     src4 = rasterio.open(f'{scene_dir}/{name}__Red.tif')
#     #src5 = rasterio.open(f'{scene_dir}/{name}__Red_Edge1.tif')
#     #src6 = rasterio.open(f'{scene_dir}/{name}__Red_Edge2.tif')
#     #src7 = rasterio.open(f'{scene_dir}/{name}__Red_Edge3.tif')
#     #src8 = rasterio.open(f'{scene_dir}/{name}__NIR_Broad.tif')
#     src8A = rasterio.open(f'{scene_dir}/{name}__NIR_Narrow.tif')
#     src11 = rasterio.open(f'{scene_dir}/{name}__SWIR1.tif')
#     src12 = rasterio.open(f'{scene_dir}/{name}__SWIR2.tif')
#     #src9 = rasterio.open(f'{scene_dir}/{name}__Water_Vapor.tif')
#     #src10 = rasterio.open(f'{scene_dir}/{name}__Cirrus.tif')
    
#     bands = []
#     for src in (src2,src3,src4,src8A,src11,src12): # Set the tuple to match desired bands
    
#         # Scale and clip reflectances
#         band = np.clip(src.read(1)[y1:y2, x1:x2]*scale, 0, 1)
#         bands.append(band)
        
#     bands = np.array(bands)
        
#     ### Locate missing data and apply to the mask file
#     missing_data = bands[0] == 0
#     subset_data[0, missing_data] = -1
    
#     # Plot the new mask    
#     fig, (ax1, ax2) = pp.subplots(ncols=2)
#     ax1.imshow(data)
#     ax1.set_title('Whole Scene Mask', fontsize=14, fontweight='roman')
#     ax2.imshow(subset_data[0,:,:])
#     ax2.set_title('Subsetted', fontsize=14, fontweight='roman')
#     pp.savefig(f'{svdir}/mask_plots/{mask.split("/")[-1].replace("tif", "jpg")}')
#     pp.close()
        
#     # Save the new mask
#     out_meta = src.meta
#     out_meta.update({'driver':'GTiff', 'height':subset_data.shape[1],
#         'width':subset_data.shape[2]})
#     with rasterio.open(f'{svdir}/masks/subsetted_{npixels}x{npixels}_{mask.split("/")[-1]}', 'w', **out_meta) as dest:
#         dest.write(subset_data)
        
#     ### Save the scene tiff
#     out_meta.update({'driver':'GTiff', 'height':bands.shape[1],
#         'width':bands.shape[2], 'count':bands.shape[0], 'dtype':bands.dtype})
#     save_name = f'{svdir}/scenes/subsetted_{npixels}x{npixels}_{name}_merged.tif'
#     with rasterio.open(save_name, 'w', **out_meta) as dest:
#         dest.write(bands)

Adjusted code using rasterio mask with crop = True (works, needs cleanup)

In [ ]:
# # Open the geojson file and extract the bounding box coordinates
# with open(geojson_file) as f:
#     aoi_data = json.load(f)
#     coords = aoi_data['features'][0]['geometry']['coordinates'][0]

# aoi_src = geopandas.read_file(geojson_file)
# aoi_src.iloc[0].geometry
# print(aoi_src.crs)

In [90]:
geojson_files = sorted(glob(geojson_dir+'*.geojson'))
print(geojson_files)

['C:/Users/mcecil/CGA/CDL/geojson\\aoi.geojson']


In [91]:
# Load the HLS tiles and place there coordinates into a numpy array for processing later
fiona.drvsupport.supported_drivers['KML'] = 'rw'
tile_src = geopandas.read_file(kml_file, driver='KML')
tile_name = []
tile_x = []
tile_y = []
for tile_ind in range(tile_src.shape[0]):
    tile_name.append(tile_src.iloc[tile_ind].Name)
    tile_x.append(tile_src.iloc[tile_ind].geometry.centroid.x)
    tile_y.append(tile_src.iloc[tile_ind].geometry.centroid.y)
tile_name = np.array(tile_name)
tile_x = np.array(tile_x)
tile_y = np.array(tile_y)
tile_src = pd.concat([tile_src, tile_src.bounds], axis = 1)
#del tile_src
tile_src.head(5)

,Name,Description,geometry,minx,miny,maxx,maxy
0,01CCV,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -7...,-180.0,-73.064633,180.0,-72.012478
1,01CDH,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,-180.0,-83.835334,180.0,-82.796720
2,01CDJ,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,-180.0,-82.939452,180.0,-81.906947
3,01CDK,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,-180.0,-82.044055,180.0,-81.016439
4,01CDL,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,-180.0,-81.148070,180.0,-80.124456


In [122]:
from rasterio.windows import Window
import rasterio
from rasterio.mask import mask as m
import json
import pyproj
  
## load image index 
image_index = pd.read_csv(image_index_file)
#print(image_index)
    
# Scale value for HLS S30 bands
scale = 0.0001

for geojson in geojson_files:
    aoi_src = geopandas.read_file(geojson) ## this could be a list
    aoi_src = pd.concat([aoi_src, aoi_src.bounds], axis = 1)
    aoi_src['centroid_x'] = aoi_src.centroid.x
    aoi_src['centroid_y'] = aoi_src.centroid.y
    print(aoi_src)
    aoi_row = aoi_src.iloc[0]
   
    aoi_x = float(aoi_row.centroid_x)
    aoi_y = float(aoi_row.centroid_y)
        
        
    # Identify closest tile
    s = (tile_x-aoi_x)**2+(tile_y-aoi_y)**2
    tname = tile_name[np.argmin(s)]
   # print(s)
   # print(tname)


    candidate_images_df = image_index.loc[(image_index['tile'] == tname) & (image_index['convert_to_tif'] == True)]
    candidate_images = candidate_images_df['image_id'].tolist()
    print(candidate_images)
    for image in candidate_images:
        ### Comment out unwanted bands
        #src1 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Coastal_Aerosol__5070.tif')
        src2 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Blue__5070.tif')
        src3 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Green__5070.tif')
        src4 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Red__5070.tif')
        #src5 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Red_Edge1__5070.tif')
        #src6 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Red_Edge2__5070.tif')
        #src7 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Red_Edge3__5070.tif')
        #src8 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__NIR_Broad__5070.tif')
        src8A = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__NIR_Narrow__5070.tif')
        src11 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__SWIR1__5070.tif')
        src12 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__SWIR2__5070.tif')
        #src9 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Water_Vapor__5070.tif')
        #src10 = rasterio.open(f'{tiff_dir_rpj}/{image}/{image}__Cirrus.__5070tif')
        
      #  src_template = 
       # print(src_template.crs)
        print('aoi crs')
        print(aoi_src.crs)
        print('raster crs')
        print(src2.crs)
        aoi_shape = aoi_src.to_crs(src2.crs).iloc[0]
        print(aoi_shape)

        
        ## read in blue band for transformation

        out_image_template, out_transform_template = rasterio.mask.mask(src2, [aoi_shape.geometry], crop=True)
        print(out_image_template.shape)
        with rasterio.open(src2) as src:
            meta = src.meta.copy()
            band = np.clip(out_image_template*scale, 0, 1)[0]
            print(band.shape)
        bands = [band]

C:\Users\mcecil\AppData\Local\Temp\ipykernel_10040\3392016860.py:17: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  aoi_src['centroid_x'] = aoi_src.centroid.x
C:\Users\mcecil\AppData\Local\Temp\ipykernel_10040\3392016860.py:18: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  aoi_src['centroid_y'] = aoi_src.centroid.y


                                            geometry       minx       miny   
0  POLYGON ((-98.46590 28.48216, -98.46590 28.491... -98.492844  28.482162  \

        maxx       maxy  centroid_x  centroid_y  
0 -98.465904  28.491892  -98.479374   28.487027  
['HLS.S30.T14RNS.2020107.v1.4', 'HLS.S30.T14RNS.2020110.v1.4', 'HLS.S30.T14RNS.2020115.v1.4', 'HLS.S30.T14RNS.2020117.v1.4', 'HLS.S30.T14RNS.2020122.v1.4', 'HLS.S30.T14RNS.2020127.v1.4', 'HLS.S30.T14RNS.2020130.v1.4', 'HLS.S30.T14RNS.2020135.v1.4', 'HLS.S30.T14RNS.2020140.v1.4', 'HLS.S30.T14RNS.2020145.v1.4', 'HLS.S30.T14RNS.2020147.v1.4', 'HLS.S30.T14RNS.2020155.v1.4', 'HLS.S30.T14RNS.2020157.v1.4', 'HLS.S30.T14RNS.2020160.v1.4', 'HLS.S30.T14RNS.2020165.v1.4', 'HLS.S30.T14RNS.2020167.v1.4', 'HLS.S30.T14RNS.2020172.v1.4', 'HLS.S30.T14RNS.2020185.v1.4', 'HLS.S30.T14RNS.2020187.v1.4', 'HLS.S30.T14RNS.2020190.v1.4', 'HLS.S30.T14RNS.2020192.v1.4', 'HLS.S30.T14RNS.2020195.v1.4', 'HLS.S30.T14RNS.2020197.v1.4', 'HLS.S30.T14RNS.2020200.v1.4'

ProjError: Error creating Transformer from CRS.

In [83]:

    
    
    
for mask in masks[0:10]:
    print(mask)
    mask = mask.replace('\\', '/')
    # Read data
    src = rasterio.open(mask)
    mask_image, mask_transform = rasterio.mask.mask(rasterio.open(mask), [aoi_shape.geometry], crop=True)
    src_data = mask_image
    print(src_data)
    print(src_data.shape)
    
    subset_data = src_data
    
   # print(src.shape)
    data = src_data[0] #.read(1)
    
    # Skip if no burn scar
    if (np.sum(data == 1) == 0):
        print('skipped')
        continue
    
      
    ### Grab matching scene files
    name = mask.replace('\\', '/').split("/")[-1].replace(".mask.tif", "")
    print(name)

    scene_dir = f'{tiff_dir}{name}'
    print(scene_dir)
   # scene_dir = f'{scdir}'

  
    
# Convert burn scar shape to proper geometry
    aoi_shape = aoi_src.to_crs(src.crs).iloc[0]

    print(aoi_shape)
        # Build the mask
    #out_image, out_transform = rasterio.mask.mask(src, [aoi_shape.geometry], crop=False)
    
   ## read in blue band for transformation
  #  with rasterio.open(src2) as src:

    out_image_template, out_transform_template = rasterio.mask.mask(rasterio.open(src2), [aoi_shape.geometry], crop=True)
    print(out_image_template.shape)
    with rasterio.open(src2) as src:
        meta = src.meta.copy()
  #  meta = src.meta.copy()
  #  m(src, [aoi_shape.geometry], crop=True)
        # Scale and clip reflectances
        band = np.clip(out_image_template*scale, 0, 1)[0]
       # bands.append(band)
    print(band.shape)
    bands = [band]
    
    for src in (src3,src4,src8A,src11,src12): # Set the tuple to match desired bands
        out_image, out_transform = rasterio.mask.mask(rasterio.open(src), [aoi_shape.geometry], crop=True)
        # Scale and clip reflectances
        band = np.clip(out_image*scale, 0, 1)[0]
        bands.append(band)
        
    bands = np.array(bands)
    print(bands)
    print(bands.shape)
    ### Locate missing data and apply to the mask file
    missing_data = bands[0] == 0
  #  subset_data[0, missing_data] = -1
    
    # Update the metadata with the new dimensions and transform
    meta.update({"driver": "GTiff",
                 "height": out_image_template.shape[1],
                 "width": out_image_template.shape[2],
                 "transform": out_transform_template})

    # Save the cropped image to a new .tif file
   # with rasterio.open("output_cropped.tif", "w", **meta) as dest:
    #    dest.write(out_image)
    
    

    # Plot the new mask    
    fig, (ax1, ax2) = pp.subplots(ncols=2)
    ax1.imshow(data)
    ax1.set_title('Whole Scene Mask', fontsize=14, fontweight='roman')
    ax2.imshow(subset_data[0,:,:])
    ax2.set_title('Subsetted', fontsize=14, fontweight='roman')
    pp.savefig(f'{svdir}/mask_plots/{mask.split("/")[-1].replace(".tif", "_2.jpg")}')
    pp.close()
        
    # Save the new mask
   # out_meta = src.meta.copy()
   # out_meta.update({'driver':'GTiff', 'height':subset_data.shape[1],
   #     'width':subset_data.shape[2]})
    with rasterio.open(f'{svdir}/masks/subsetted_{npixels}x{npixels}_{mask.split("/")[-1].replace(".tif", "_2.tif")}', 'w', **meta) as dest:
        dest.write(subset_data)
    
                 
    meta.update({"count": 6,
                'dtype': 'float32'})
    ### Save the scene tiff
    out_meta.update({'driver':'GTiff', 'height':bands.shape[1],
        'width':bands.shape[2], 'count':bands.shape[0], 'dtype':bands.dtype})
    save_name = f'{svdir}/scenes/subsetted_{npixels}x{npixels}_{name}_merged_2.tif'
    print(meta)
    print(bands.shape)
    with rasterio.open(save_name, 'w', **meta) as dest:
        dest.write(bands)

C:/Users/mcecil/CGA/CDL/masks\HLS.S30.T14RNS.2020002.v1.4.mask.tif
[[[-1000 -1000 -1000 ... -1000 -1000 -1000]
  [    0     0     0 ...     0     0 -1000]
  [    0     0     0 ...     0     0 -1000]
  ...
  [    0     0     0 ...     0     0 -1000]
  [    0     0     0 ...     0     0 -1000]
  [    0     0     0 ...     0     0 -1000]]]
(1, 37, 89)
skipped
C:/Users/mcecil/CGA/CDL/masks\HLS.S30.T14RNS.2020005.v1.4.mask.tif
[[[-1000 -1000 -1000 ... -1000 -1000 -1000]
  [    1     1     1 ...     1     1 -1000]
  [    1     1     1 ...     1     1 -1000]
  ...
  [    1     1     1 ...     1     1 -1000]
  [    1     1     1 ...     1     1 -1000]
  [    1     1     1 ...     1     1 -1000]]]
(1, 37, 89)
HLS.S30.T14RNS.2020005.v1.4
C:/Users/mcecil/CGA/CDL/tif/HLS.S30.T14RNS.2020005.v1.4
geometry    POLYGON ((552278.436 3150731.967, 552273.643 3...
Name: 0, dtype: geometry


RasterioIOError: C:/Users/mcecil/CGA/CDL/tif/HLS.S30.T14RNS.2020005.v1.4/HLS.S30.T14RNS.2020005.v1.4__Blue.tif: No such file or directory

In [ ]:
from osgeo import gdal

raster_bounds = src.bounds
print(raster_bounds)
upper_left_x = 699934.584491
upper_left_y = 6169364.0093
lower_right_x = 700160.946739
lower_right_y = 6168703.00544
window = (upper_left_x,upper_left_y,lower_right_x,lower_right_y)

gdal.Translate('output_crop_raster.tif', 'input_raster.tif', projWin = window)